In [1]:
# Scientific libraries		              
import numpy as np		              
import scipy as sp		              
import pandas as pd		              
import json		              
import missingno as msno		              

# Loading Plotting Modules		              
import matplotlib		              
import matplotlib.pyplot as plt		              
import seaborn as sns		              
%matplotlib inline		              
import chart_studio.plotly as py		              
import plotly.figure_factory as ff		              
import plotly.graph_objects as go		              
import plotly.express as px		              
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot		              
init_notebook_mode(connected=True)		              
		              
import os		              
import gc

In [2]:
# Setting Data Frame Options                      
pd.set_option('display.max_rows', 40)                      
pd.set_option('display.max_columns', 50)                      
pd.set_option('display.width', 100)                      
pd.set_option('display.max_colwidth', 100)                      

# Setting Plot Configuration                      
sns.set(rc={'figure.figsize':(19,11)}, style = 'white')

In [4]:
test = pd.read_csv('./outputs/imputed_test_mf.csv')                
m, n = test.shape
print('Data Frame: {} x {}'.format(m,n))                
test.head()

Data Frame: 890 x 79


,galactic year,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,"Intergalactic Development Index (IDI), Rank",Population using at least basic drinking-water services (%),Population using at least basic sanitation services (%),Gross capital formation (% of GGP),"Population, total (millions)","Population, urban (%)","Mortality rate, under-five (per 1,000 live births)","Mortality rate, infant (per 1,000 live births)",Old age dependency ratio (old age (65 and older) per 100 creatures (ages 15-64)),"Population, ages 15–64 (millions)","Population, ages 65 and older (millions)","Life expectancy at birth, male (galactic years)","Life expectancy at birth, female (galactic years)","Population, under age 5 (millions)",Young age (0-14) dependency ratio (per 100 creatures ages 15-64),"Adolescent birth rate (births per 1,000 female creatures ages 15-19)",...,"Estimated gross galactic income per capita, male","Estimated gross galactic income per capita, female",Rural population with access to electricity (%),Domestic credit provided by financial sector (% of GGP),"Population with at least some secondary education, female (% ages 25 and older)","Population with at least some secondary education, male (% ages 25 and older)",Gross fixed capital formation (% of GGP),"Remittances, inflows (% of GGP)",Population with at least some secondary education (% ages 25 and older),Intergalactic inbound tourists (thousands),"Gross enrolment ratio, primary (% of primary under-age population)","Respiratory disease incidence (per 100,000 people)",Interstellar phone subscriptions (per 100 people),"Interstellar Data Net users, total (% of population)",Current health expenditure (% of GGP),"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings,Gender Inequality Index (GII),galaxy,Private galaxy capital flows (% of GGP),"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total"
0,1007012.0,0.868360,72.982208,19198.499040,0.700413,16.145285,11.113581,0.719579,0.825915,181.824530,76.357958,71.796137,71.977811,819.245976,72.269765,113.394831,75.983899,12.851245,426.589757,59.793499,73.206969,79.051548,80.144699,87.255770,77.624057,...,NaN,NaN,102.306960,NaN,NaN,NaN,75.084941,16.383957,NaN,92046.182973,129.047095,578.672958,70.150003,39.261197,11.067889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Andromeda Galaxy (M31),32.163414,NaN
1,1007012.0,0.772696,73.590698,18345.350181,0.684405,12.167872,7.083681,0.650826,0.505962,193.187549,79.263683,71.723281,37.085742,849.398477,53.465398,139.090298,88.145153,14.152255,496.779346,57.583581,68.021732,74.795340,73.158109,89.776748,124.050371,...,19268.820517,12931.719986,99.189863,77.742674,54.263456,69.496155,38.331603,9.435519,62.266114,92843.780990,131.408452,429.753270,104.910863,34.341097,6.029590,0.673039,0.714560,0.976994,147.451303,164.334975,6.673241,0.687616,Andromeda I,30.668839,5.784397
2,1007012.0,0.941941,81.973436,26708.317145,0.925563,18.175905,13.909442,0.960263,0.937407,74.678611,122.988241,136.685519,34.865297,712.914221,93.116852,59.810528,34.960991,31.242181,435.976780,54.971127,78.814709,87.318457,54.861556,51.049333,72.544281,...,46130.422627,29804.266408,133.499902,112.410524,110.028144,112.483100,29.294321,9.716045,124.671861,98471.178149,117.291927,216.424101,152.699066,89.310138,10.114774,0.975232,0.949269,1.078944,91.321283,80.725039,16.251653,0.467704,Andromeda III,25.648367,3.641324
3,1007012.0,1.079857,89.351427,54738.000742,1.016415,19.927563,12.917355,1.085718,1.010280,65.576254,113.617318,144.223620,27.907457,994.869334,93.992349,50.614834,37.408702,39.063774,452.063178,68.716581,87.959292,90.227666,83.641251,43.62177

In [16]:
# RMSE Train: 0.001800391617117943
# RMSE Valid: 0.052504038023260875

preds = pd.read_csv('../modeling/outputs/y_pred_lgbm_mf_0525val.csv')
preds['opt_pred'] = 0
preds['eei'] = test['existence expectancy index']
preds['p_increase'] = -np.log(preds['y_pred'] + 0.01) + 3
preds['p_increase2'] = preds['p_increase'] ** 2
preds

,Index,y_pred,opt_pred,eei,p_increase,p_increase2
0,0,0.041248,0,0.868360,5.971077,35.653766
1,1,0.046107,0,0.772696,5.880488,34.580138
2,2,0.114819,0,0.941941,5.080892,25.815461
3,3,0.177866,0,1.079857,4.672029,21.827850
4,4,0.224418,0,1.079557,4.450649,19.808277
...,...,...,...,...,...,...
885,885,0.040630,0,0.850609,5.983221,35.798930
886,886,0.053313,0,0.808027,5.759666,33.173747
887,887,0.062798,0,1.012209,5.620070,31.585187
888,888,0.040166,0,0.704373,5.992420,35.909100


### Optimization

We will try to maximize sum of likely increase index for all galaxies, which for one galaxy is described as follows:

    Likely increase in the Index = extra energy * Potential for increase in the Index ** 2 / 1000

with respect to the following constraints.

* in total there are 50000 zillion DSML available for allocation
* no galaxy should be allocated more than 100 zillion DSML or less than 0 zillion DSML
* galaxies with low existence expectancy index below 0.7 should be allocated at least 10% of the total energy available

In [65]:
res_df

,Index,y_pred,opt_pred,eei,p_increase,p_increase2
7,7,0.052085,389.177869,0.650576,5.779257,33.399809
8,8,0.041013,412.267513,0.652121,5.975682,35.708774
12,12,0.040515,413.440076,0.651697,5.985485,35.826030
27,27,0.040134,414.346017,0.676239,5.993048,35.916624
36,36,0.041331,411.524837,0.649197,5.969464,35.634506
...,...,...,...,...,...,...
885,885,0.040630,19.380846,0.850609,5.983221,35.798930
886,886,0.053313,22.006028,0.808027,5.759666,33.173747
887,887,0.062798,23.594588,1.012209,5.620070,31.585187
888,888,0.040166,19.270675,0.704373,5.992420,35.909100


In [127]:
def total_likely_increase(opt_pred, p_sqr):
    return sum((opt_pred * p_sqr**2) / 1000)


def optimize(data_frame, alpha_low=10, alpha_high=1, steps=100, tol=1):
    max_energy = 50000
    galaxy_lower_bound = 0
    galaxy_upper_bound = 100
    low_end_upper_eei = 0.7
    low_end_galaxy_min_energy = 5000
    
    totals = [0]
    low_end_energies = [0]
    data_frame['opt_pred'] = max_energy / data_frame.shape[0] - 1
    low_end = data_frame.loc[data_frame['eei'] < low_end_upper_eei].copy()
    high_end = data_frame.loc[data_frame['eei'] >= low_end_upper_eei].copy()
    
    for _ in range(steps):
        low_end_energy = low_end['opt_pred'].sum()
        high_end_energy = high_end['opt_pred'].sum()
        total_energy = low_end_energy + high_end_energy
        
        if total_energy>(max_energy - tol):
            break
            
        if low_end_energy == low_end_energies[-1]:
            break
        else:
            low_end_energies.append(low_end_energy)
            
        low_end_increase = total_likely_increase(low_end['opt_pred'], low_end['p_increase2'])
        high_end_increase = total_likely_increase(high_end['opt_pred'], high_end['p_increase2'])
        total_increase = low_end_increase + high_end_increase
        
        if total_increase<totals[-1]:
            break
        
        print(f"Sum of Likely Increase: {total_increase:.3f}\tLow end energy: {low_end_energy:.3f}\t"
              f"Total energy: {total_energy:.3f}")
        totals.append(total_increase)
        
        dLE = alpha_low*low_end['p_increase2']/1000
        dHE = alpha_high*high_end['p_increase2']/1000
        
        low_end['opt_pred'] = (low_end['opt_pred'] + dLE).clip(0,100)
        high_end['opt_pred'] = (high_end['opt_pred'] - dHE).clip(0,100) 
    
    resulting_df = low_end.append(high_end, ignore_index=False)
    resulting_df.reset_index(drop=True, inplace=True)
    totals.remove(0)
    return totals, resulting_df


In [144]:
sums, res_df = optimize(preds, alpha_low=1.5, alpha_high=0.1, steps=1000)

Sum of Likely Increase: 47260.185	Low end energy: 3697.045	Total energy: 49110.000
Sum of Likely Increase: 47262.301	Low end energy: 3700.632	Total energy: 49111.106
Sum of Likely Increase: 47264.418	Low end energy: 3704.218	Total energy: 49112.212
Sum of Likely Increase: 47266.534	Low end energy: 3707.805	Total energy: 49113.319
Sum of Likely Increase: 47268.650	Low end energy: 3711.392	Total energy: 49114.425
Sum of Likely Increase: 47270.766	Low end energy: 3714.978	Total energy: 49115.531
Sum of Likely Increase: 47272.882	Low end energy: 3718.565	Total energy: 49116.637
Sum of Likely Increase: 47274.999	Low end energy: 3722.152	Total energy: 49117.744
Sum of Likely Increase: 47277.115	Low end energy: 3725.738	Total energy: 49118.850
Sum of Likely Increase: 47279.231	Low end energy: 3729.325	Total energy: 49119.956
Sum of Likely Increase: 47281.347	Low end energy: 3732.912	Total energy: 49121.062
Sum of Likely Increase: 47283.463	Low end energy: 3736.498	Total energy: 49122.169
Sum 

Sum of Likely Increase: 47567.035	Low end energy: 4217.114	Total energy: 49270.404
Sum of Likely Increase: 47569.151	Low end energy: 4220.701	Total energy: 49271.510
Sum of Likely Increase: 47571.267	Low end energy: 4224.288	Total energy: 49272.617
Sum of Likely Increase: 47573.383	Low end energy: 4227.874	Total energy: 49273.723
Sum of Likely Increase: 47575.499	Low end energy: 4231.461	Total energy: 49274.829
Sum of Likely Increase: 47577.616	Low end energy: 4235.048	Total energy: 49275.935
Sum of Likely Increase: 47579.732	Low end energy: 4238.634	Total energy: 49277.041
Sum of Likely Increase: 47581.848	Low end energy: 4242.221	Total energy: 49278.148
Sum of Likely Increase: 47583.964	Low end energy: 4245.808	Total energy: 49279.254
Sum of Likely Increase: 47586.080	Low end energy: 4249.394	Total energy: 49280.360
Sum of Likely Increase: 47588.197	Low end energy: 4252.981	Total energy: 49281.466
Sum of Likely Increase: 47590.313	Low end energy: 4256.568	Total energy: 49282.573
Sum 

Sum of Likely Increase: 47859.070	Low end energy: 4712.077	Total energy: 49423.064
Sum of Likely Increase: 47861.187	Low end energy: 4715.663	Total energy: 49424.171
Sum of Likely Increase: 47863.303	Low end energy: 4719.250	Total energy: 49425.277
Sum of Likely Increase: 47865.419	Low end energy: 4722.837	Total energy: 49426.383
Sum of Likely Increase: 47867.535	Low end energy: 4726.424	Total energy: 49427.489
Sum of Likely Increase: 47869.651	Low end energy: 4730.010	Total energy: 49428.596
Sum of Likely Increase: 47871.768	Low end energy: 4733.597	Total energy: 49429.702
Sum of Likely Increase: 47873.884	Low end energy: 4737.184	Total energy: 49430.808
Sum of Likely Increase: 47876.000	Low end energy: 4740.770	Total energy: 49431.914
Sum of Likely Increase: 47878.116	Low end energy: 4744.357	Total energy: 49433.021
Sum of Likely Increase: 47880.232	Low end energy: 4747.944	Total energy: 49434.127
Sum of Likely Increase: 47882.349	Low end energy: 4751.530	Total energy: 49435.233
Sum 

Sum of Likely Increase: 48159.571	Low end energy: 5221.386	Total energy: 49580.150
Sum of Likely Increase: 48161.687	Low end energy: 5224.973	Total energy: 49581.256
Sum of Likely Increase: 48163.804	Low end energy: 5228.559	Total energy: 49582.362
Sum of Likely Increase: 48165.920	Low end energy: 5232.146	Total energy: 49583.468
Sum of Likely Increase: 48168.036	Low end energy: 5235.733	Total energy: 49584.575
Sum of Likely Increase: 48170.152	Low end energy: 5239.319	Total energy: 49585.681
Sum of Likely Increase: 48172.268	Low end energy: 5242.906	Total energy: 49586.787
Sum of Likely Increase: 48174.385	Low end energy: 5246.493	Total energy: 49587.893
Sum of Likely Increase: 48176.501	Low end energy: 5250.079	Total energy: 49589.000
Sum of Likely Increase: 48178.617	Low end energy: 5253.666	Total energy: 49590.106
Sum of Likely Increase: 48180.733	Low end energy: 5257.253	Total energy: 49591.212
Sum of Likely Increase: 48182.849	Low end energy: 5260.840	Total energy: 49592.318
Sum 

Sum of Likely Increase: 48460.072	Low end energy: 5730.695	Total energy: 49737.235
Sum of Likely Increase: 48462.188	Low end energy: 5734.282	Total energy: 49738.341
Sum of Likely Increase: 48464.304	Low end energy: 5737.869	Total energy: 49739.448
Sum of Likely Increase: 48466.421	Low end energy: 5741.455	Total energy: 49740.554
Sum of Likely Increase: 48468.537	Low end energy: 5745.042	Total energy: 49741.660
Sum of Likely Increase: 48470.653	Low end energy: 5748.629	Total energy: 49742.766
Sum of Likely Increase: 48472.769	Low end energy: 5752.215	Total energy: 49743.873
Sum of Likely Increase: 48474.885	Low end energy: 5755.802	Total energy: 49744.979
Sum of Likely Increase: 48477.002	Low end energy: 5759.389	Total energy: 49746.085
Sum of Likely Increase: 48479.118	Low end energy: 5762.975	Total energy: 49747.191
Sum of Likely Increase: 48481.234	Low end energy: 5766.562	Total energy: 49748.297
Sum of Likely Increase: 48483.350	Low end energy: 5770.149	Total energy: 49749.404
Sum 

Sum of Likely Increase: 48760.573	Low end energy: 6240.005	Total energy: 49894.320
Sum of Likely Increase: 48762.689	Low end energy: 6243.591	Total energy: 49895.427
Sum of Likely Increase: 48764.805	Low end energy: 6247.178	Total energy: 49896.533
Sum of Likely Increase: 48766.921	Low end energy: 6250.765	Total energy: 49897.639
Sum of Likely Increase: 48769.038	Low end energy: 6254.351	Total energy: 49898.745
Sum of Likely Increase: 48771.154	Low end energy: 6257.938	Total energy: 49899.852
Sum of Likely Increase: 48773.270	Low end energy: 6261.525	Total energy: 49900.958
Sum of Likely Increase: 48775.386	Low end energy: 6265.111	Total energy: 49902.064
Sum of Likely Increase: 48777.502	Low end energy: 6268.698	Total energy: 49903.170
Sum of Likely Increase: 48779.619	Low end energy: 6272.285	Total energy: 49904.277
Sum of Likely Increase: 48781.735	Low end energy: 6275.871	Total energy: 49905.383
Sum of Likely Increase: 48783.851	Low end energy: 6279.458	Total energy: 49906.489
Sum 

In [145]:
sums = pd.DataFrame(sums, columns=['sum_likely_increase'])
px.scatter(sums,
           x=sums.index,
          y='sum_likely_increase')

In [146]:
res_df.opt_pred.sum()

49999.41275294995

In [147]:
res_df.drop(['eei', 'p_increase', 'p_increase2'], axis=1, inplace=True)
res_df.rename(columns= {
    "Index": "index",
    "y_pred": "pred"
}, inplace=True)
res_df

,index,pred,opt_pred
0,7,0.052085,95.459945
1,8,0.041013,98.244556
2,12,0.040515,98.385967
3,27,0.040134,98.495224
4,36,0.041331,98.154990
...,...,...,...
885,885,0.040630,52.301541
886,886,0.053313,52.512606
887,887,0.062798,52.640326
888,888,0.040166,52.292684


In [148]:
res_df.to_csv('../../submissions/pending/lgb_val0525.csv', index=False)